<a href="https://colab.research.google.com/github/shalinisaini/CS60092_9A_18CS60R47/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
SEED =42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


In [174]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:

url = 'https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Test_5K.csv'
df1 = pd.read_csv(url, sep='\t')

In [0]:
url = 'https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Train_20K.csv'
df2 = pd.read_csv(url, sep='\t')

In [0]:

train_data = df2['text'].astype(str).tolist()
train_label = df2['label'].astype(int)
test_data = df1['text'].astype(str).tolist()
test_label = df1['label'].astype(int)

train_data = [x.lower() for x in train_data]
test_data = [x.lower() for x in test_data]


 


In [0]:
def preprocess(data):
  stopword = set(stopwords.words('english'))
  dt2=[]
  for i in data:
    words = i.split(" ")
    dt =[]
    for j in words:
      if(j not in stopword and not j.isdigit() and len(j)>1):
        dt.append(j)
    dt2.append(dt)
  return dt2

              

In [0]:

train_data = preprocess(train_data)
test_data = preprocess(test_data)

In [101]:
print(train_data[0])

['john', 'waters', 'given', 'us', 'genuinely', 'enjoyable', 'film.', 'certainly', 'without', 'shocking', 'waters-esque', 'moments,', 'tamer', 'older', 'culty', 'stuff,', '"pink', 'flamingoes".', '"pecker"', 'harkens', 'back', "john's", 'early', 'mainstream', 'stage', 'reminds', 'viewer', 'kind', 'humor', 'evident', '"polyester".', 'overall,', 'really', 'fun', 'comedy', 'great', 'moments!']


In [102]:
print(len(train_data))

17999


In [180]:
#for task6--

ans=[]
for line in train_data:
    #print(line)
    arr = line
    tmp=[]
    for i in range(len(arr)-1):
        tmp.append([arr[i], arr[i+1]])
        #del(tmp[-1:])
    #word_to_ix.append(ans)
    ans.append(tmp)
print(train_data[0])


['john', 'waters', 'given', 'us', 'genuinely', 'enjoyable', 'film.', 'certainly', 'without', 'shocking', 'waters-esque', 'moments,', 'tamer', 'older', 'culty', 'stuff,', '"pink', 'flamingoes".', '"pecker"', 'harkens', 'back', "john's", 'early', 'mainstream', 'stage', 'reminds', 'viewer', 'kind', 'humor', 'evident', '"polyester".', 'overall,', 'really', 'fun', 'comedy', 'great', 'moments!']


In [152]:
ans1=[]
for a in ans:
  for b in a:
    ans1.append(b[0]+" "+b[1])
print(ans1[0])

john waters


In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  print(dataset)
  for sent in dataset:
      for word in sent:
          if word not in word_to_ix:
              word_to_ix[word] = 1
          else:
              word_to_ix[word] = word_to_ix[word]+1
  return word_to_ix

In [0]:
#for task6
def generate_word_idsbg(dataset):
  word_to_bix = {}
  print(dataset)
  for sent in dataset:
      for word in sent:
          if word not in word_to_bix:
              word_to_bix[word] = 1
          else:
              word_to_bix[word] = word_to_bix[word]+1
  a={}
  for word in word_to_bix:
    if word_to_bix[word]>10:
      a[word]=word_to_bix[word]
  
  return a

In [131]:
word_to_ix = generate_word_ids(train_data)
print(len(word_to_ix))
NUM_LABELS = 2



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



31854


In [182]:
#for task6--
word_to_bix = generate_word_idsbg(ans1)
print(len(word_to_bix))
vocab_size_b = len(word_to_bix)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



111


In [109]:
list=[]
for i in word_to_ix:
  if(word_to_ix[i]>10):
    list.append(i)
word_to_ix={}
for i in list:
  word_to_ix[i] = len(word_to_ix)
  
VOCAB_SIZE = len(word_to_ix)
print(VOCAB_SIZE)

19704


In [0]:

  

class Task6(nn.Module):
  def __init__(self, num_labels, vocab_size_b):
    super(Task6, self).__init__()
    self.dp=nn.Dropout(0.01)
    self.dp1=nn.Dropout(0.01)
    self.dp2=nn.Dropout(0.01)
    self.lin = nn.Linear(vocab_size_b, 30)
    self.lin4 = nn.Linear(30,20)
    self.lin5 = nn.Linear(20,10)
    self.lin6 = nn.Linear(10, num_labels)
    
    
    
  def forward(self, x):
    x=(self.lin(x))
    x=self.dp(x)
    x=(self.lin4(x))
    x=self.dp1(x)
    x=(self.lin5(x))
    x=self.dp2(x)
    return F.softmax(self.lin6(x))
  
  
  #task1a ---
class Task1a(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin2 = nn.Linear(50, num_labels)
    
  def forward(self, x):
     return F.softmax(self.lin2(self.lin(x)))
    
    
  #task1b ---
class Task1b(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin2 = nn.Linear(100, num_labels)
  def forward(self, x):
    return F.softmax(self.lin2(self.lin(x)))
    
    
  #task1c ---
class Task1c(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 200)
    self.lin2 = nn.Linear(200, num_labels)
  def forward(self, x):
    return F.softmax(self.lin2(self.lin(x)))
    
    
  #task1d ---
class Task1d(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 500)
    self.lin2 = nn.Linear(500, num_labels)
  def forward(self, x):
    return F.softmax(self.lin2(self.lin(x)))
  
 
    
  #task2a ---
class Task2a(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 10)
    self.lin2 = nn.Linear(10, 10)
    self.lin3 = nn.Linear(10, num_labels)
  def forward(self, x):
    return F.softmax(self.lin3(self.lin2(self.lin(x))))
  
  #task2b ---
class Task2b(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 20)
    self.lin2 = nn.Linear(20, 10)
    self.lin3 = nn.Linear(10, num_labels)
  def forward(self, x):
     return F.softmax(self.lin3(self.lin2(self.lin(x))))
    
   #task2c ---
class Task2c(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 30)
    self.lin2 = nn.Linear(30, 30)
    self.lin3 = nn.Linear(30, num_labels)
  def forward(self, x):
    return F.softmax(self.lin3(self.lin2(self.lin(x))))
    
   #task2d ---
class Task2d(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin2 = nn.Linear(50, 50)
    self.lin3 = nn.Linear(50, num_labels)
  def forward(self, x):
    return F.softmax(self.lin3(self.lin2(self.lin(x))))
    
   #task2e ---
class Task2d(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin2 = nn.Linear(100, 50)
    self.lin3 = nn.Linear(50, num_labels)
  def forward(self, x):
    return F.softmax(self.lin3(self.lin2(self.lin(x))))
    
  #task3a----
class Task3a(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin4 = nn.Linear(100,50)
    self.lin5 = nn.Linear(50,10)
    self.lin6 = nn.Linear(10, num_labels)
  def forward(self, x):
      return F.softmax(self.lin6(self.lin5(self.lin4(self.lin(x)))))
  
  
   #task3b-----  
class Task3b(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, 200)
    self.lin4 = nn.Linear(200,100)
    self.lin5 = nn.Linear(100,10)
    self.lin6 = nn.Linear(10, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin6(self.lin5(self.lin4(self.lin(x)))))
  
  
  #task 4a----
class Task4a(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.4)
    self.lin = nn.Linear(vocab_size, 30)
    self.lin4 = nn.Linear(30,20)
    self.lin5 = nn.Linear(20,10)
    self.lin6 = nn.Linear(10, num_labels)
  def forward(self, x):
    x=self.lin(x)
    x=self.dp(x)
    x=self.lin4(x)
    x=self.dp(x)
    x=self.lin5(x)
    x=self.dp(x)
    return F.softmax(self.lin6(x))
  
    
  
 #task4b----
class Task4b(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.1)
    self.dp1=nn.Dropout(0.2)
    self.lin = nn.Linear(vocab_size, 100)
    self.lin4 = nn.Linear(100,100)
    self.lin6 = nn.Linear(100, num_labels)
  def forward(self, x):
    x=F.relu(self.lin(x))
    x=self.dp(x)
    x=F.relu(self.lin4(x))
    x=self.dp1(x)
    return F.softmax(self.lin6(x))
  
   
 #task4c----
class Task4c(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.1)
    self.dp1=nn.Dropout(0.2)
    self.lin = nn.Linear(vocab_size, 100)
    self.lin4 = nn.Linear(100,10)
    self.lin6 = nn.Linear(10, num_labels)
  def forward(self, x):
    x=F.relu(self.lin(x))
    x=self.dp(x)
    x=F.relu(self.lin4(x))
    x=self.dp1(x)
    return F.softmax(self.lin6(x))
    
 #task5a----  
class Task5a(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.2)
    self.dp1=nn.Dropout(0.3)
    self.dp2=nn.Dropout(0.4)
    self.lin = nn.Linear(vocab_size, 30)
    self.lin4 = nn.Linear(30,20)
    self.lin5 = nn.Linear(20,10)
    self.lin6 = nn.Linear(10, num_labels)
  def forward(self, x):
    x=F.relu(self.lin(x))
    x=self.dp(x)
    x=F.relu(self.lin4(x))
    x=self.dp1(x)
    x=F.relu(self.lin5(x))
    x=self.dp2(x)
    return F.softmax(self.lin6(x))

    
 #task5b----  
class Task5b(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.2)
    self.dp1=nn.Dropout(0.3)
    self.dp2=nn.Dropout(0.4)
    self.lin = nn.Linear(vocab_size, 30)
    self.lin4 = nn.Linear(30,20)
    self.lin5 = nn.Linear(20,10)
    self.lin6 = nn.Linear(10, num_labels)
  def forward(self, x):
    x=F.tanh(self.lin(x))
    x=self.dp(x)
    x=F.tanh(self.lin4(x))
    x=self.dp1(x)
    x=F.tanh(self.lin5(x))
    x=self.dp2(x)
    return F.softmax(self.lin6(x))

 #task5c----  
class Task5c(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.2)
    self.dp1=nn.Dropout(0.3)
    self.dp2=nn.Dropout(0.4)
    self.lin = nn.Linear(vocab_size, 30)
    self.lin4 = nn.Linear(30,20)
    self.lin5 = nn.Linear(20,10)
    self.lin6 = nn.Linear(10, num_labels)
  def forward(self, x):
    x=F.sigmoid(self.lin(x))
    x=self.dp(x)
    x=F.sigmoid(self.lin4(x))
    x=self.dp1(x)
    x=F.sigmoid(self.lin5(x))
    x=self.dp2(x)
    return F.softmax(self.lin6(x))
  
 #task7a
class Task7a(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.01)
    self.dp1=nn.Dropout(0.01)
    self.dp2=nn.Dropout(0.01)
    self.lin = nn.Linear(vocab_size, 30)
    self.lin4 = nn.Linear(30,20)
    self.lin5 = nn.Linear(20,10)
    self.lin6 = nn.Linear(10, num_labels)
     
    
  def forward(self, x):
    x=(self.lin(x))
    x=self.dp(x)
    x=(self.lin4(x))
    x=self.dp1(x)
    x=(self.lin5(x))
    x=self.dp2(x)
    return F.softmax(self.lin6(x))
  
  #task7b
class Task7b(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.01)
    self.dp1=nn.Dropout(0.01)
    self.dp2=nn.Dropout(0.01)
    self.lin = nn.Linear(vocab_size, 30)
    self.lin4 = nn.Linear(30,20)
    self.lin5 = nn.Linear(20,10)
    self.lin6 = nn.Linear(10, num_labels)
     
    
  def forward(self, x):
    x=(self.lin(x))
    x=self.dp(x)
    x=(self.lin4(x))
    x=self.dp1(x)
    x=(self.lin5(x))
    x=self.dp2(x)
    return F.softmax(self.lin6(x))
  
   #task7c
class Task7c(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.dp=nn.Dropout(0.01)
    self.dp1=nn.Dropout(0.01)
    self.dp2=nn.Dropout(0.01)
    self.lin = nn.Linear(vocab_size, 30)
    self.lin4 = nn.Linear(30,20)
    self.lin5 = nn.Linear(20,10)
    self.lin6 = nn.Linear(10, num_labels)
     
    
  def forward(self, x):
    x=(self.lin(x))
    x=self.dp(x)
    x=(self.lin4(x))
    x=self.dp1(x)
    x=(self.lin5(x))
    x=self.dp2(x)
    return F.softmax(self.lin6(x))

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            c=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.FloatTensor([label_to_ix[label]])
    #for task7B == return torch.FloatTensor([label_to_ix[label]])
  
  
  



In [0]:
#for task6--
def make_bow_vector_b(sentence, word_to_bix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_bix))
    for word in sentence:
        if word not in word_to_bix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            c=1
        else:
            vec[word_to_bix[word]]+=1
    return vec.view(1, -1)

In [0]:
bow1a = Task1a(NUM_LABELS, VOCAB_SIZE).cuda()


In [0]:
bow1b = Task1b(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow1c =  Task1c(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow1d =  Task1d(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow2a =  Task2a(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow2b =  Task2b(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow2c =  Task2c(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow2d =  Task2d(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow3a =  Task3a(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow3b =  Task3b(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow4a =  Task4a(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow4b =  Task4b(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow4c =  Task4c(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow5a =  Task5a(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow5b =  Task5b(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow5c =  Task5c(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow6 = Task6(NUM_LABELS, vocab_size_b)


In [0]:
bow7a =  Task7a(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow7b =  Task7b(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
bow7c =  Task7c(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
def accuracy(data,label):
  count=0
  tp =0
  tn=0
  fp=0
  fn=0
  for idx in range(len(data)):
      bow_vec = Variable(make_bow_vector(data[idx], word_to_ix)).cuda()
      labels = Variable(torch.LongTensor(label[idx])).cuda()
      logprobs = bow(bow_vec).reshape(-1)
      val,pred = logprobs.max(0)
      #print(pred,label[idx]+1)
      if(pred == label[idx] and pred==1):
        count+=1
        tp+=1
        
      elif(pred == label[idx] and pred==0):
        tn+=1
      
      elif(pred !=label[idx] and pred==1):
        fp+=1
        
      elif(pred !=label[idx] and pred==0):
        fn+=1
  
  precision =(tp)/(tp+fp)
  recall=(tp/(tp+fn))
  print("precision {}".format(float(tp)/(tp+fp)))
  print("recall {}".format(float(tp/(tp+fn))))
  print("F1 Score {}".format(float(2*((precision*recall)/(precision+recall)))))
  print("accuracy {}".format(float(count/len(data))))
        


In [0]:
def accuracy_bg(data,label):
  count=0
  tp =0
  tn=0
  fp=0
  fn=0
  for idx in range(len(data)):
      bow_vec = Variable(make_bow_vector(data[idx], word_to_bix))
      labels = Variable(torch.FloatTensor(label[idx]))
      logprobs = bow6(bow_vec).reshape(-1)
      val,pred = logprobs.max(0)
      #print(pred,label[idx]+1)
      if(pred == label[idx] and pred==1):
        count+=1
        tp+=1
        
      elif(pred == label[idx] and pred==0):
        tn+=1
      
      elif(pred !=label[idx] and pred==1):
        fp+=1
        
      elif(pred !=label[idx] and pred==0):
        fn+=1
  
  precision =(tp)/(tp+fp)
  recall=(tp/(tp+fn))
  print("precision {}".format(float(tp)/(tp+fp)))
  print("recall {}".format(float(tp/(tp+fn))))
  print("F1 Score {}".format(float(2*((precision*recall)/(precision+recall)))))
  print("accuracy {}".format(float(count/len(data))))
        

In [0]:
#task7a---
# define a loss function and an optimizer
loss_function = nn.CrossEntropyLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [0]:
#task7b---
# define a loss function and an optimizer
loss_function = nn.MSELoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [0]:
#task7c---
# define a loss function and an optimizer
loss_function = nn.HingeEmbeddingLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [0]:
# the training loop
#task7b---
for epoch in range(5):
    print("epoch",epoch)
    for idx in range(len(train_data)):
        # get the training data
        bow.zero_grad()
        #print(train_data[idx])
        #print(train_label[idx])
        bow_vec = Variable(make_bow_vector(train_data[idx], word_to_ix)).cuda()
        #label  = Variable(torch.FloatTensor([train_label[idx]])).cuda()
        label  = Variable(torch.FloatTensor([train_label[idx]])).cuda()
        #print(bow_vec,label,train_label[idx])
        probs = bow(bow_vec) # forward pass
        #print(probs)
        loss = loss_function(probs, label)
        #opt.zero_grad()
        loss.backward()
        opt.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    accuracy(train_data,train_label)


In [0]:
#task 6


# the training loop
for epoch in range(5):
    print("epoch",epoch)
    for idx in range(len(train_data)):
        # get the training data
        bow6.zero_grad()
        #print(train_data[idx])
        #print(train_label[idx])
        bow_vec = Variable(make_bow_vector_b(train_data[idx], word_to_bix))
        label  = Variable(torch.FloatTensor([train_label[idx]]))
        #for task7b-- label  = Variable(torch.FloatTensor([train_label[idx]])).cuda()
        #print(bow_vec,label,train_label[idx])
        probs = bow6(bow_vec) # forward pass
        #print(probs)
        loss = loss_function(probs, label)
        #opt.zero_grad()
        loss.backward()
        opt.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    accuracy_bg(train_data,train_label)


In [170]:
accuracy(test_data,test_label)

RuntimeError: ignored

In [172]:
torch.save(bow6.cpu(),'Task6.mdl')
from google.colab import files
files.download("Task6.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task6. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
